In [1]:
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

import pandas as pd
import numpy as np



Using TensorFlow backend.


In [2]:
# Импортируем данные
train = pd.read_csv('./data/digit/train.csv')
labels = train.ix[:,0].values.astype('int32')
X_train = (train.ix[:,1:].values).astype('float32')
X_test = (pd.read_csv('./data/digit/test.csv').values).astype('float32')



In [3]:
X_train.shape, X_test.shape

((42000, 784), (28000, 784))

### препроцессинг

In [4]:
# создаем категориальную матрицу из лейблов
Y_train = np_utils.to_categorical(labels) 

In [5]:

# пре-процессинг - нормируем данные 

# делим на максимальное 
scale = np.max(X_train)
X_train /= scale
X_test /= scale


# отнимаем среднее

mean = np.std(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]
nclasses = Y_train.shape[1]



In [6]:
input_dim, nclasses

(784, 10)

In [7]:
# инициируем DDMLP сеть
model = Sequential()
model.add(Dense(128, input_dim=input_dim, init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128, init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(nclasses, init='lecun_uniform'))
model.add(Activation('relu'))

# комплируем модель
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
__________

In [9]:
## Фитим сгенерированную модель и проверяем скоринг на валидации 0.01
model.fit(X_train, Y_train, epochs=30, batch_size=64, validation_split=0.01, verbose=1)



Train on 41580 samples, validate on 420 samples
Epoch 1/30
41580/41580 [==============================] - 4s - loss: 0.0288 - acc: 0.8429 - val_loss: 0.0128 - val_acc: 0.9310
Epoch 2/30
41580/41580 [==============================] - 4s - loss: 0.0151 - acc: 0.9292 - val_loss: 0.0106 - val_acc: 0.9405
Epoch 3/30
41580/41580 [==============================] - 4s - loss: 0.0117 - acc: 0.9406 - val_loss: 0.0074 - val_acc: 0.9571
Epoch 4/30
41580/41580 [==============================] - 4s - loss: 0.0101 - acc: 0.9476 - val_loss: 0.0078 - val_acc: 0.9524
Epoch 5/30
41580/41580 [==============================] - 4s - loss: 0.0090 - acc: 0.9525 - val_loss: 0.0073 - val_acc: 0.9571
Epoch 6/30
41580/41580 [==============================] - 4s - loss: 0.0082 - acc: 0.9556 - val_loss: 0.0069 - val_acc: 0.9500
Epoch 7/30
41580/41580 [==============================] - 4s - loss: 0.0077 - acc: 0.9579 - val_loss: 0.0058 - val_acc: 0.9690
Epoch 8/30
41580/41580 [==============================] - 4s - 

In [10]:
## предсказываем
predictions = model.predict_classes(X_test, verbose=1)




27008/28000 [===========================>..] - ETA: 0s

In [12]:
def write_preds(preds, f):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": predictions}).to_csv(f, index=False, header=True)

write_preds(predictions, "cv_predictions.txt")


In [ ]:
### kaggle 0.96657